### Canvas Creation from Enrichr

In [40]:
import pandas as pd 
import numpy as np
import pickle
import json
import requests
import math
import matplotlib
import uuid
import urllib
from textwrap import dedent
from IPython.core.display import display, HTML
from string import Template
from random import seed, randint
from operator import itemgetter

Below this is arbitrary definitions for these variables, their actual values will be selected in the Appyter (so DON'T ADD BELOW CELL TO APPYTER)

In [41]:
all_libraries = ['ChEA_2016',
    'ENCODE_and_ChEA_Consensus_TFs_from_ChIP-X',
    'ENCODE_Histone_Modifications_2015',
    'ENCODE_TF_ChIP-seq_2015',
    'Epigenomics_Roadmap_HM_ChIP-seq',
    'Genome_Browser_PWMs',
    'TargetScan_microRNA_2017',
    'TF-LOF_Expression_from_GEO',
    'Transcription_Factor_PPIs',
    'TRANSFAC_and_JASPAR_PWMs',
    'TRRUST_Transcription_Factors_2019']

In [42]:
def library_processing(library_index):
    # processes library data
    raw_library_data = []
    library_data = []

    with urllib.request.urlopen('https://amp.pharm.mssm.edu/Enrichr/geneSetLibrary?mode=text&libraryName=' + all_libraries[library_index]) as f:
        for line in f.readlines():
                raw_library_data.append(line.decode("utf-8").split("\t\t"))
    name = []
    gene_list = []

    for i in range(len(raw_library_data)):
        name += [raw_library_data[i][0]]
        raw_genes = raw_library_data[i][1].split("\t")
        gene_list += [raw_genes[:-1]]

    # library_data: a 2D list where the first element is the name and the second element is a list of genes 
    # associated with that name
    library_data = [list(a) for a in zip(name, gene_list)]

    jaccard_indices = []
    indices = []

    for gene_set in library_data:
        intersection = [value for value in gene_set[1] if value in genes]
        index = len(intersection)/(len(gene_set[1]) + len(genes))
        jaccard_indices += [[gene_set[0], index]]
        indices += [round(index, 5)]

    # determine the dimensions of the canvas
    x_dimension = math.ceil(math.sqrt(len(indices)))
    y_dimension = math.ceil(math.sqrt(len(indices)))

    # zip name, gene_list, indices, and blank list for neighbor score then add dummy entries to the zipped list
    num_hex = x_dimension*y_dimension
    neighbor_score = [0.0] * len(name)
    anneal_list = list(zip(name, gene_list, indices, neighbor_score))

    unzipped_anneal_list = [name, gene_list, indices, neighbor_score]

    # add "dummy" hexagons so the rectangular shape is filled
    for i in range(len(name), num_hex):
        anneal_list += [('', [], 0.0, 0.0)]

    return anneal_list, x_dimension, y_dimension

def unzip_list(anneal_list):
    unzipped_list = zip(*anneal_list)
    return list(unzipped_list)

In [43]:
def find_neighbors(ind, x_dimension, y_dimension):
    # returns a list of the indices of the neighbors of the index given
    
    neighbors = []
    num_hex = x_dimension * y_dimension

    if 0 <= ind <= x_dimension-1:
        # top row (inc. corners)
        if ind == 0:
            # top left corner
            neighbors = [num_hex-1, num_hex-x_dimension, x_dimension-1, 2*x_dimension-1, ind+1, ind+x_dimension]
        elif ind == (x_dimension-1):
            # top right corner
            neighbors = [ind-1, ind+x_dimension, 0, ind+x_dimension-1, num_hex-2, num_hex-1]
        else:
            # non-corner top row
            neighbors = [ind-1, ind+1, ind+x_dimension, ind+num_hex-x_dimension-1, 
            ind+num_hex-x_dimension, ind+x_dimension-1]

    elif (num_hex - x_dimension) <= ind <= num_hex -1:
        if ind == (num_hex-x_dimension):
            # bottom left corner
            neighbors = [ind+1, ind-x_dimension, ind-x_dimension+1, 0, 1, num_hex-1]
        elif ind == (num_hex-1):
            # bottom right corner
            neighbors = [ind-1, ind-x_dimension, ind-x_dimension+1, 0, x_dimension-1,
            num_hex-2*x_dimension]
        else:
            # non-corner bottom row
            neighbors = [ind-1, ind+1, ind-x_dimension, ind-x_dimension+1, ind-num_hex+x_dimension,
            ind-num_hex+x_dimension+1]
    elif ind % y_dimension == 0 and (ind/y_dimension)%2 == 1:
        # "inner" left edge (not top or bottom row)
        neighbors = [ind+x_dimension-1, ind+1, ind-x_dimension, ind-x_dimension+1, ind+x_dimension, 
        ind+x_dimension+1]
    elif ind % y_dimension == 0 and (ind/y_dimension)%2 == 0:
        # "outer" left edge (not top or bottom row)
        neighbors = [ind-1, ind+1, ind+x_dimension, ind+2*x_dimension-1, ind-x_dimension, 
        ind+x_dimension-1]
    elif (ind+1) % y_dimension == 0 and ((ind+1)/y_dimension)%2 == 0:
        # "outer" right edge (not top or bottom row)
        neighbors = [ind-1, ind+1, ind-x_dimension, ind-x_dimension+1, ind+x_dimension, 
        ind-2*x_dimension+1]
    elif (ind+1) % y_dimension == 0 and ((ind+1)/y_dimension)%2 == 1:
        # "inner" right edge (not top or bottom row)
        neighbors = [ind-1, ind-x_dimension-1, ind-x_dimension, ind-x_dimension+1, ind+x_dimension, 
        ind+x_dimension-1]
    else:
        # middle
        neighbors = [ind-1, ind+1, ind-x_dimension, ind-x_dimension+1, ind+x_dimension, 
        ind+x_dimension+1]
    return neighbors

In [44]:
# initially find fitness
def find_fitness(anneal_list, x_dimension, y_dimension):
    fitness = 0
    for i in range(len(anneal_list)):
        neighbors = find_neighbors(i, x_dimension, y_dimension)
        sum_neighbor_score = 0
        for index in neighbors:
            intersection = [value for value in anneal_list[index][1] if value in anneal_list[i][1]]
            if len(anneal_list[index][1]) + len(anneal_list[i][1]) != 0:
                jaccard = len(intersection)/(len(anneal_list[index][1]) + len(anneal_list[i][1]))
            else:
                jaccard = 0.0
            sum_neighbor_score += jaccard
        hex_list = list(anneal_list[i])
        hex_list[3] = sum_neighbor_score
        hex_tuple = tuple(hex_list)
        anneal_list[i] = hex_tuple
        fitness += sum_neighbor_score
    return fitness, anneal_list

# take indices of swapped hexagons
def find_swapped_fitness(anneal_list, swapped_a, swapped_b, old_fitness, x_dimension, y_dimension):
    neighbors_a = find_neighbors(swapped_a, x_dimension, y_dimension)
    neighbors_b = find_neighbors(swapped_b, x_dimension, y_dimension)
    hexagons_to_update = [swapped_a, swapped_b] + neighbors_a + neighbors_b
    anneal_copy = anneal_list.copy()

    new_fitness = 0
    # Recalculate scores for all hexagons that need updating
    for hex in hexagons_to_update:

        # subtract out the swapped neighbor fitnesses because they are changing 
        old_fitness -= anneal_copy[hex][3]

        neighbors = find_neighbors(hex, x_dimension, y_dimension)
        sum_neighbor_score = 0
        for index in neighbors:
            intersection = [value for value in anneal_copy[index][1] if value in anneal_copy[hex][1]]
            if len(anneal_copy[index][1]) + len(anneal_copy[hex][1]) != 0:
                jaccard = len(intersection)/(len(anneal_copy[index][1]) + len(anneal_copy[hex][1]))
            else:
                jaccard = 0.0
            sum_neighbor_score += jaccard
        hex_list = list(anneal_copy[hex])
        hex_list[3] = sum_neighbor_score
        hex_tuple = tuple(hex_list)
        anneal_copy[hex] = hex_tuple
        new_fitness += sum_neighbor_score
        
    return old_fitness + new_fitness, anneal_copy

In [45]:
def annealing(anneal_list, steps, old_fitness, x_dimension, y_dimension):
    num_hex = x_dimension * y_dimension
    # returns unzipped list
    for i in range(steps):
        index_a = randint(0, num_hex-1)
        index_b = randint(0, num_hex-1)
        anneal_list[index_a], anneal_list[index_b] = anneal_list[index_b], anneal_list[index_a]
        new_fitness, new_anneal_list = find_swapped_fitness(anneal_list, index_a, index_b, old_fitness, x_dimension, y_dimension)

        if new_fitness <= old_fitness:
            # swap back
            anneal_list[index_a], anneal_list[index_b] = anneal_list[index_b], anneal_list[index_a]
        else:
            # finalize the swap by resetting old_fitness and changing anneal_list
            old_fitness = new_fitness
            anneal_list = new_anneal_list
    return anneal_list

In [46]:
for library_index in range(len(all_libraries)):
    anneal_list, x_dimension, y_dimension = library_processing(library_index)
    anneal_list = annealing(anneal_list, 100000, find_fitness(anneal_list, x_dimension, y_dimension)[0], x_dimension, y_dimension)
    with open('Annealed-Libaries/' + all_libraries[library_index] + '.txt', 'wb+') as f:
        pickle.dump(anneal_list, f)

KeyboardInterrupt: 